In [1]:
import sys
sys.path.append('../')

In [2]:
import game.agent as ag
from game.run import run
from game.simulator import load_simulator_yarn, GraphSimulator
from pathlib import Path
from tqdm.auto import trange
import pandas as pd
# import mpire
import torch

## Description

This game only has one personality trait, which is safety (no risk).

Therefore, a 1 implies a safe action (without risk), and a 0 implies a risky action.

In [3]:
# n_runs = int(1e4)
n_runs = int(100)
seed = None

## Simulator

In [4]:
simulator = load_simulator_yarn('../yarnScripts', graph_file_sfx='_graph_transformed.pickle')

## Agents

In [5]:
# label_predictor=ag.TorchLabelPredictor(
#             model_path=Path('./saved_bert/200_[20]_[30]_1_False_bert-base-multilingual-cased_0.001_adamw_8_max_val_mcc_False_False_100'),
#             use_cpu=False,
#         )

def get_linear_agent(a: torch.Tensor):
    """
    Gets an agent with a BehavioralDecisionMaker that has a linear function

    :param torch.Tensor a: matrix A that characterizes the behavior profile
    """
    return ag.LabelDecisorAgent(
        label_predictor=ag.GraphLabelLoader(
            simulator=simulator,
        ),
        decision_maker=ag.BehavioralDecisionMaker(
            weight_funcs=[
                lambda x: (a*x).sum()
            ],
            memory_steps=a.shape[0]-1,
            seed=None,
            deterministic=False,
        )
    )

In [6]:
def get_random_agent():
    return ag.RandomAgent(None)


# def get_r_agent():
#     return ag.LabelDecisorAgent(
#         label_predictor=ag.GraphLabelLoader(
#             simulator=simulator,
#         ),
#         decision_maker=ag.RDecisionMaker(
#             rand=0.4,
#             seed=None,
#         )
#     )


def get_risky_agent():
    return get_linear_agent(torch.as_tensor([
        [2]
    ], dtype=torch.float))


def get_risky_agent():
    return get_linear_agent(torch.as_tensor([
        [-2]
    ], dtype=torch.float))


def get_risky_ntm_agent():
    return get_linear_agent(torch.as_tensor([
        [0.4],
        [-0.4],
        [-0.3],
    ], dtype=torch.float))


# def get_mc_agent():
#     return ag.LabelDecisorAgent(
#         label_predictor=ag.GraphLabelLoader(
#             simulator=simulator,
#         ),
#         decisor=ag.MarkovChainAgent(
#             rand=0.4,
#             seed=None,
#         )
#     )

In [7]:
# def get_random_agent():
#     return ag.RandomAgent(None)


# # def get_r_agent():
# #     return ag.LabelDecisorAgent(
# #         label_predictor=ag.GraphLabelLoader(
# #             simulator=simulator,
# #         ),
# #         decision_maker=ag.RDecisionMaker(
# #             rand=0.4,
# #             seed=None,
# #         )
# #     )


# def get_risky_agent():
#     return ag.LabelDecisorAgent(
#         label_predictor=ag.GraphLabelLoader(
#             simulator=simulator,
#         ),
#         decision_maker=ag.BehavioralDecisionMaker(
#             weight_funcs=[
#                 lambda x: x
#             ],
#             memory_steps=0,
#             seed=None,
#         )
#     )


# def get_risky_agent():
#     return ag.LabelDecisorAgent(
#         label_predictor=ag.GraphLabelLoader(
#             simulator=simulator,
#         ),
#         decision_maker=ag.BehavioralDecisionMaker(
#             weight_funcs=[
#                 lambda x: x
#             ],
#             memory_steps=0,
#             seed=None,
#         )
#     )


# def get_risky_ntm_agent():
#     return ag.LabelDecisorAgent(
#         label_predictor=ag.GraphLabelLoader(
#             simulator=simulator,
#         ),
#         decision_maker=ag.BehavioralDecisionMaker(
#             weight_funcs=[
#                 lambda x: 0.5*x,
#                 lambda x: -0.3*x,
#                 lambda x: -0.3*x,
#             ],
#             memory_steps=3,
#             seed=None,
#         )
#     )


# # def get_mc_agent():
# #     return ag.LabelDecisorAgent(
# #         label_predictor=ag.GraphLabelLoader(
# #             simulator=simulator,
# #         ),
# #         decisor=ag.MarkovChainAgent(
# #             rand=0.4,
# #             seed=None,
# #         )
# #     )

## Run

### Random Agent

In [8]:
df = run(get_random_agent, simulator, n_runs)

100%|██████████| 100/100 [00:00<00:00, 4941.80it/s]


In [9]:
# df.to_csv('endings.csv')

In [10]:
df['endings']

,run,title,kind
0,0,MaloAmigos,0
1,1,MaloSilencio,0
2,2,MaloAmigos,0
3,3,MaloSilencio,0
4,4,MaloSilencio,0
...,...,...,...
95,95,MaloAmanda,0
96,96,MaloSilencio,0
97,97,MaloPadres,0
98,98,MaloSilencio,0


In [11]:
t = df['decisions'].loc[df['decisions']['qid'] != -1,:]
t

,run,num,qid,choice,label
0,0,0,1.0,1,NaN
1,0,1,2.0,1,1.0
3,0,3,3.0,0,NaN
5,0,5,4.0,0,1.0
6,0,6,5.0,1,1.0
...,...,...,...,...,...
1983,99,9,8.0,0,0.0
1985,99,11,9.0,1,0.0
1986,99,12,10.0,1,0.0
1987,99,13,11.0,1,0.0


In [12]:
# t.pivot(index='run', columns='qid', values='label')['label']

In [13]:
df['decisions'].groupby(['qid']).mean()
# diferencia entre estos valores y los otros nos dice si tiene predilección por label o no

,run,num,choice,label
qid,,,,
-1.0,49.568878,11.973214,0.000000,NaN
1.0,49.500000,0.000000,0.540000,NaN
2.0,49.500000,1.000000,0.610000,0.610000
3.0,49.500000,3.000000,0.500000,NaN
4.0,49.500000,5.000000,0.580000,0.420000
5.0,49.500000,6.000000,0.590000,0.590000
6.0,52.237288,7.000000,0.593220,0.593220
7.0,49.500000,7.590000,0.490000,0.490000
8.0,53.285714,8.551020,0.489796,0.489796


In [14]:
rand = df['decisions'].groupby(['num']).mean()
rand['m'] = 0

### Risky Agent

In [15]:
df = run(get_risky_agent, simulator, n_runs)

  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\vibal\PycharmProjects\text-games\notebooks\..\game\agent.py:238: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  self._p = F.softmax((pred * self.w[None, :]).sum(1)) # (actions, labels) -> (actions)
100%|██████████| 100/100 [00:00<00:00, 248.57it/s]


In [16]:
df['endings']

,run,title,kind
0,0,MaloSilencio,0
1,1,MaloSilencio,0
2,2,MaloSilencio,0
3,3,MaloSilencio,0
4,4,MaloSilencio,0
...,...,...,...
95,95,MaloSilencio,0
96,96,MaloSilencio,0
97,97,MaloSilencio,0
98,98,MaloSilencio,0


In [ ]:
df['decisions'].groupby(['num']).mean()
# diferencia entre estos valores y los otros nos dice si tiene predilección por label o no

In [ ]:
df['decisions'].groupby(['num']).mean()['choice'].plot()

In [ ]:
risk = df['decisions'].groupby(['num']).mean()
risk['m'] = 1

### Risky But Not So Much Agent

In [ ]:
df = run(get_risky_ntm_agent, simulator, n_runs)

In [ ]:
df['endings']

In [ ]:
df['decisions'].groupby(['num']).mean()
# diferencia entre estos valores y los otros nos dice si tiene predilección por label o no

In [ ]:
df['decisions'].groupby(['num']).mean()['choice'].plot()

In [ ]:
risk_ntm = df['decisions'].groupby(['num']).mean()
risk_ntm['m'] = 1

## Data Analysis

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.concat([rand,risk])
df = df.reset_index()
df = df.dropna()

df_train,df_test = train_test_split(df, test_size=0.3)

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(df_train.loc[:,df.columns!='m'], df_train['m'])

lr = LogisticRegression()
lr.fit(df_train.loc[:,df.columns!='m'], df_train['m'])

In [ ]:
dt.score(df_test.loc[:,df.columns!='m'], df_test['m'])

In [ ]:
lr.score(df_test.loc[:,df.columns!='m'], df_test['m'])